<a href="https://colab.research.google.com/github/alostmathematician/ISTA-366/blob/main/%E2%80%9CNB5_ISTA_322_SQL_intermediate_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL

Woooooo, I know you're all excited for more SQL work!  This week we're going to go deeper into the SQL world and start doing more filtering, aggregations, grouping, joins, and mixes of them all.  

You've already been introduced to all these topics and learned how to code them in Python.  A fair question is: why do this in SQL vs. just doing it locally?  Well, two reasons.  For one, your data might just be too darn big to bring in locally, so it's more efficient to do this on the SQL server.  The other might be that even if your data is small it might be more efficient to do the bulk of the work on the SQL server and then bring it in locally.  Doing that would mean less data transfer and ultimately faster speed.  

There are not strict rules regarding what you should do via SQL vs. Python, but I'd say it makes sense to do most of the simple selections, filtering, aggregations via SQL.  If you have to do more complicated work or a lot of exploration, it may pay to bring it in locally and work there.  But again, each situation is different so I'm not going to try and make a rulebook.  Use your best judgement.  

What are we going to learn this lesson?  

* GROUP BY - We'll first do some GROUP BY operations in order to aggregate some basic information about our movies
* HAVING - HAVING is a way to filter on the output of your GROUP BY.  For example, if you want to get the average rating for each movie, but only return those that have an average greater than 3
* JOIN - You've seen JOIN before, and now we're going to do them in SQL.  They allow us to bring together different tables into one.  
* Subqueries - Subqueries are, well, queries within queries.  You might want to get a query that meets a condition in one table, and then return only values from a different table that match that condition.  It's essentially query inception.  
* Odds and ends - We'll also learn some other random bits such as how to do math across columns, rename columns, expand our functions, make some new ones, etc.  

## Libraries and functions
We'll start by bringing in our usual libraries.  

We're also going to bring in our functions but we're going to modify them a bit to give us more info.  We're also going to make a couple new ones to check what tables are in our database and what column names are within our tables.

### Connection function

This one we'll keep the same!  It just returns a connection called `conn` and a cursor call `cur`.

In [1]:
import psycopg2
import pandas as pd

In [2]:
# Make our connection/cursor function
AWS_host_name = "moviesdb.cp9xqenk8fzu.us-east-1.rds.amazonaws.com"
AWS_dbname = "moviesdb"
AWS_user_name = "postgres"
AWS_password = "ista322moviesdb"

def get_conn_cur(): # define function name and arguments (there aren't any)
  # Make a connection
  conn = psycopg2.connect(
    host=AWS_host_name,
    database=AWS_dbname,
    user=AWS_user_name,
    password=AWS_password,
    port='5432')

  cur = conn.cursor()   # Make a cursor after

  return(conn, cur)   # Return both the connection and the cursor

In [3]:
conn, cur = get_conn_cur()
conn

<connection object at 0x7f84c1940b80; dsn: 'user=postgres password=xxx dbname=moviesdb host=moviesdb.cp9xqenk8fzu.us-east-1.rds.amazonaws.com port=5432', closed: 0>

### Table name function

Let's make a function that allows us to quickly check what table names are present.  This is actually just the same function we used before to run a query string, but this time we're embedding just the one query string in the function so it only gets the table names.  This way you don't have to rewrite the string and then call it in the old function.

As you saw in the last lesson, our database contains something called an information_schema which contains, well, information about our database.  Within that you can get the tables, hence calling `infomation_schema.tables` and selecting just their names.  Feel free to check out the other information in `information_tables` here: https://www.postgresql.org/docs/9.1/information-schema.html!

In [4]:
def get_table_names():
  conn, cur = get_conn_cur() # get connection and cursor

  # query to get table names
  table_name_query = """SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public' """

  cur.execute(table_name_query) # execute
  my_data = cur.fetchall() # fetch results

  cur.close() #close cursor
  conn.close() # close connection

  return(my_data) # return your fetched results

Now we can run just the function `get_table_names()` whenever we want to remember what tables we have.  This is a lot easier and faster than rewriting that query statement each time and running it!

In [5]:
# Run it!h
get_table_names()

[('movies',), ('ratings',), ('movie_sales',)]

### Column names function

Let's also make a function to tell us what the column names are in a given table.  This is a touch tricker as we need to specify the table name as an argument and then insert that inside of a string.  We'll need to do just a touch of string formatting to make it work.  

`%s` as a placeholder operator that allows you to pass a specified value into a string.  You specify what value you want to place there by using `%` and then what you want to insert.  Here's a simple example:

```
>>> print('hello my name is %s' % 'dan')
hello my name is dan
```
There are easier ways to do the simple example above, but the format we just used will come in handy later when things get more complicated.  

For now, we're going to use it to allow us to sub in the name of the table we need columns names from.  

Let's make a function called `get_column_names`.  This function should take a table name as an argument.  It should then insert that name into our query string that gets the columns from the information_schema.

I'll first show you how it'll get added to our string, then we'll make the function.

In [6]:
# First, watch what happens when you assign a table name to an object and then add it to the string
tn = 'test_table_name'
column_name_query =  """SELECT column_name FROM information_schema.columns WHERE table_name = '%s' """ %tn
column_name_query # Note how it just added the object name at the end to make a functional query

"SELECT column_name FROM information_schema.columns WHERE table_name = 'test_table_name' "

In [7]:
# Just for comparisons watch what happens if we didn't use %s and instead tried to insert directly.
tn = 'test_table_name'
column_name_query =  """SELECT column_name FROM information_schema.columns WHERE table_name = tn """
column_name_query # Note how it didn't put the object 'tn' inside but just left it as a string.

'SELECT column_name FROM information_schema.columns WHERE table_name = tn '

In [8]:
# Now make a function that uses this approach.

def get_column_names(table_name): # arguement of table_name
  conn, cur = get_conn_cur() # get connection and cursor

  # Now select column names while inserting the table name into the WERE
  column_name_query =  """SELECT column_name FROM information_schema.columns
       WHERE table_name = '%s' """ %table_name

  cur.execute(column_name_query) # exectue
  my_data = cur.fetchall() # store

  cur.close() # close
  conn.close() # close

  return(my_data) # return

In [37]:
get_column_names(table_name = 'ratings')

[('userid',), ('movieid',), ('rating',), ('timestamp',)]

In [9]:
# Test it out!
get_column_names(table_name = 'movie_sales')

[('movieid',), ('domestic',), ('international',)]

Great!  These functions will be really helpful when you're joining multiple tables or doing aggregations.  They allow for you to in just one line get the info you need vs. writing a query each time.  

### Expanding our query function

In last week's lesson, we had our query function where we could specify a query string and it would run it while doing all the opening, closing, etc.  One issue is that it doesn't give you the column names. That's going to be an issue later on in the lesson so let's build a bit more functionality into our `run_query` function so that it also gets the table name.  

Remember that after we exectue our query our cursor stores the resulting information in memory.  We use `fetchall()` on the cursor to get that info, but the cursor also contains other info.  Specifically, the description also contains information about each column that was returned.  We can use a list comprehension to pull out the column names.  I've added to the function below!

In [10]:
def run_query(query_string):

  conn, cur = get_conn_cur() # get connection and cursor

  cur.execute(query_string) # executing string as before

  my_data = cur.fetchall() # fetch query data as before

  # here we're extracting the 0th element for each item in cur.description
  colnames = [desc[0] for desc in cur.description]

  cur.close() # close
  conn.close() # close

  return(colnames, my_data) # return column names AND data

Let's just test this with a quick query.  Note how the response is a tuple of two items.  The first is a list of your column names.  The second is a list of tuples of the data you requested.

In [11]:
qs = """ SELECT * FROM movies
            LIMIT 3; """
run_query(qs)

(['movieid', 'title', 'genres'],
 [(1, 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'),
  (2, 'Jumanji (1995)', 'Adventure|Children|Fantasy'),
  (3, 'Grumpier Old Men (1995)', 'Comedy|Romance')])

You could also easily store the titles and data individually as they're being returned as `colnames` and `my_data` inside the function.  Watch:

In [12]:
qs = """ SELECT * FROM movies
            LIMIT 3; """
titles, query_data = run_query(qs)

In [13]:
#check titles
titles

['movieid', 'title', 'genres']

In [14]:
# check data
query_data

[(1, 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'),
 (2, 'Jumanji (1995)', 'Adventure|Children|Fantasy'),
 (3, 'Grumpier Old Men (1995)', 'Comedy|Romance')]

In [38]:
qs = """SELECT COUNT(movieid) as number_views FROM ratings
        GROUP BY movieid
        HAVING movieid IN 
        (SELECT movieid FROM movies WHERE title = 'Sudden Death (1995)');"""
        
run_query(qs)

(['number_views'], [(12,)])

## GROUP BY

Let's now jump into doing some data aggregations.  For these you need two things.  First, your GROUP BY argument which tells SQL what level you want to group your data by.  Second, you need to apply some sort of aggregation function to a column.  Postgres has the following aggregation functions built in:
* AVG() - Get the average of a column within the specified group.
* COUNT() - Count how many observations are within each group.
* MAX() - Max value within a group.
* MIN() - Min value within a group.
* SUM() - Sum of values in a column within a group

Let's start by just getting the average rating for each movie.  Let's first use our `get_column_names` function to remember what we're working with.

In [15]:
get_column_names(table_name= 'ratings')

[('userid',), ('movieid',), ('rating',), ('timestamp',)]

In [16]:
qs = """SELECT * FROM ratings LIMIT 100"""
run_query(qs)

(['userid', 'movieid', 'rating', 'timestamp'],
 [(64572, 5, 4.0, datetime.datetime(1997, 3, 27, 4, 14, 10)),
  (86695, 5, 4.0, datetime.datetime(1997, 6, 29, 16, 41, 44)),
  (151466, 5, 3.0, datetime.datetime(2017, 7, 3, 16, 13, 23)),
  (23243, 10, 4.0, datetime.datetime(1996, 4, 28, 11, 30, 51)),
  (91496, 2, 3.5, datetime.datetime(2015, 11, 7, 1, 28, 23)),
  (12966, 1, 4.5, datetime.datetime(2010, 4, 12, 7, 48, 22)),
  (123960, 7, 5.0, datetime.datetime(1999, 11, 2, 15, 1, 23)),
  (68143, 6, 2.0, datetime.datetime(2010, 5, 23, 23, 20, 20)),
  (118915, 8, 5.0, datetime.datetime(1996, 8, 14, 13, 51, 15)),
  (120763, 10, 4.0, datetime.datetime(1996, 5, 29, 18, 4, 14)),
  (118164, 1, 4.0, datetime.datetime(2007, 8, 17, 22, 35, 40)),
  (99698, 6, 5.0, datetime.datetime(2011, 4, 21, 21, 44, 41)),
  (49734, 10, 3.0, datetime.datetime(2005, 2, 17, 12, 25, 32)),
  (10327, 1, 4.0, datetime.datetime(2000, 3, 18, 21, 16, 15)),
  (157060, 1, 5.0, datetime.datetime(2000, 11, 29, 20, 15, 46)),
  (3

So, we want to get the average rating, so we'll need `AVG(rating)` in our `SELECT`.  And we want the average for each movie, which means we want to use `GROUP BY movieid`.  Let's try it.

In [17]:
# Average rating
qs = """SELECT movieid, AVG(rating) FROM ratings
          GROUP BY movieid;"""
run_query(qs)

(['movieid', 'avg'],
 [(8, 3.4285714285714284),
  (9, 3.25),
  (7, 3.2916666666666665),
  (10, 3.425925925925926),
  (1, 3.967948717948718),
  (5, 3.142857142857143),
  (4, 2.5833333333333335),
  (2, 3.2371794871794872),
  (6, 3.9375),
  (3, 2.9634146341463414)])

### Aliasing and ordering

That query worked great!  But there are some issues.  For one, notice how the column name for our average rating is just `'avg'`.  That's not very descriptive, but luckily you can rename or 'alias' columns.  You just write `as new_name` after the aggregating function.  Let's alias the column as `avg_rating`.

Similarly, the average ratings are not in any order.  Let's add an `ORDER BY` at the end to arrange by average rating. **NOTE** we need to order by the aliased column name, so in this case `avg_rating`!

In [18]:
# Average rating w/ alias and order by
qs = """SELECT movieid, AVG(rating) as avg_rating FROM ratings
          GROUP BY movieid
          ORDER BY avg_rating;"""
run_query(qs)

(['movieid', 'avg_rating'],
 [(4, 2.5833333333333335),
  (3, 2.9634146341463414),
  (5, 3.142857142857143),
  (2, 3.2371794871794872),
  (9, 3.25),
  (7, 3.2916666666666665),
  (10, 3.425925925925926),
  (8, 3.4285714285714284),
  (6, 3.9375),
  (1, 3.967948717948718)])

**TRY IT OUT**  Can you go and get a count of how many times each movie has been reviewed?  Alias the value as `number_reviews` and sort by that value.

In [19]:
# Test it out! Question 1: Write your query as the answer to the question 1.
qs = """SELECT movieid, COUNT(movieid) as number_reviews
        From ratings GROUP BY movieid ORDER BY number_reviews"""
run_query(qs)

(['movieid', 'number_reviews'],
 [(4, 6),
  (8, 7),
  (9, 12),
  (5, 35),
  (7, 36),
  (3, 41),
  (6, 48),
  (2, 78),
  (10, 81),
  (1, 156)])

### Grouping by time

Just like in python, you can group by time values.  There are a couple ways to do this, but here we're going to use a built in function called `date_part()`.  This function works a lot like datetime functions in python.  You feed it two arguments.  The first is the unit of the date you want to extract from the datetime column.  The second is the datetime column.  

In this case, our code will be `SELECT date_part('year', timestamp) as year, ....`.  This is saying "grab the year part from the datetime that's inside the timestamp column and alias it as 'year'".  

In [20]:
# review average per year
qs = """ SELECT date_part('year', timestamp) as year, AVG(rating) FROM ratings
          GROUP BY year
          ORDER BY year
          """
run_query(qs)

(['year', 'avg'],
 [(1996.0, 3.566666666666667),
  (1997.0, 3.5762711864406778),
  (1998.0, 3.5714285714285716),
  (1999.0, 3.84),
  (2000.0, 3.4285714285714284),
  (2001.0, 3.3),
  (2002.0, 3.625),
  (2003.0, 3.25),
  (2004.0, 3.857142857142857),
  (2005.0, 3.2096774193548385),
  (2006.0, 3.108695652173913),
  (2007.0, 3.4444444444444446),
  (2008.0, 3.75),
  (2009.0, 3.1666666666666665),
  (2010.0, 3.05),
  (2011.0, 3.5),
  (2012.0, 4.071428571428571),
  (2013.0, 3.7),
  (2014.0, 3.4),
  (2015.0, 3.7884615384615383),
  (2016.0, 3.525),
  (2017.0, 3.2058823529411766),
  (2018.0, 3.875),
  (2019.0, 4.375)])

## HAVING

HAVING is just essentially like WHERE but instead works on your grouped results.  For example, let's get just years where the average ratings are greater than 3.5.  

The only place it differs is that you call your aggregation again for the HAVING statement.  For example, we'd need `HAVING AVG(rating) >= 3.5` to filter only years that have an average >= 4.  Using the column alias we created in the select wouldn't work (e.g. `HAVING avg_rating >= 3.5`) due to order of SQL operations... more on that later.


In [21]:
# Average per year where avg >= 3.5
qs = """ SELECT date_part('year', timestamp) as year, AVG(rating) as avg_rating FROM ratings
          GROUP BY year
          HAVING AVG(rating) >= 3.5
          """
run_query(qs)

(['year', 'avg_rating'],
 [(2013.0, 3.7),
  (2002.0, 3.625),
  (1996.0, 3.566666666666667),
  (1998.0, 3.5714285714285716),
  (1999.0, 3.84),
  (2011.0, 3.5),
  (2019.0, 4.375),
  (2018.0, 3.875),
  (2004.0, 3.857142857142857),
  (2015.0, 3.7884615384615383),
  (2016.0, 3.525),
  (2012.0, 4.071428571428571),
  (1997.0, 3.5762711864406778),
  (2008.0, 3.75)])

**TRY IT OUT**  Get a count of how many times a movie has been reviewed.  But this time display only the movieid's and counts of movies that have been reviewed more than 50 times.

In [22]:
# Test it out! Question 2: Write your query as the answer.
qs = """SELECT movieid,COUNT(movieid) as number_reviews
        FROM ratings GROUP BY movieid
        HAVING COUNT(movieid) > 50"""
run_query(qs)

(['movieid', 'number_reviews'], [(10, 81), (1, 156), (2, 78)])

## JOIN

You've already done joins in python using `.merge()`, but now we're going to do them straight away in our database.  The concept is the exactly same in that there are different types of joins, LEFT being the most common, that are connected by a key.  

To do a JOIN you specify the type (LEFT JOIN, INNER JOIN, OUTER JOIN), then the dataframe you want to join, then ON followed by the columns to join on.  

For example, below let's join movie sales data to our dataset of movie info.  

In [23]:
# A quick look at the head of movies again
qs = """SELECT * FROM movies
          LIMIT 5;"""
run_query(qs)

(['movieid', 'title', 'genres'],
 [(1, 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'),
  (2, 'Jumanji (1995)', 'Adventure|Children|Fantasy'),
  (3, 'Grumpier Old Men (1995)', 'Comedy|Romance'),
  (4, 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance'),
  (5, 'Father of the Bride Part II (1995)', 'Comedy')])

In [24]:
# A quick look at the head of movie_sales
qs = """SELECT * FROM movie_sales
          LIMIT 5;"""
run_query(qs)

(['movieid', 'domestic', 'international'],
 [(1, 191796233, 172747718),
  (2, 100458310, 163300000),
  (3, 71518503, 0),
  (4, 66947948, 14400000),
  (5, 76578911, 0)])

We want to join the sales data *to* our movie data, so the FROM will be `FROM movies` and we'll LEFT JOIN movie_sales using, well ,`LEFT JOIN movie_sales`.

Note in order to tell it what columns you have your keys in you need to use the format `table_name.column_with_key`.  This is because we have the same column name, `movieid`, in both datasets so it needs to know which one to use from each.

In [25]:
# Join sales data to movies data
qs = """ SELECT * FROM movies
          LEFT JOIN movie_sales ON movies.movieid = movie_sales.movieid
          LIMIT 3;
          """
run_query(qs)

(['movieid', 'title', 'genres', 'movieid', 'domestic', 'international'],
 [(1,
   'Toy Story (1995)',
   'Adventure|Animation|Children|Comedy|Fantasy',
   1,
   191796233,
   172747718),
  (2, 'Jumanji (1995)', 'Adventure|Children|Fantasy', 2, 100458310, 163300000),
  (3, 'Grumpier Old Men (1995)', 'Comedy|Romance', 3, 71518503, 0)])

That worked!  Messy formatting aside, you can see that now we have a single return that contains the movie ID, title, genres, as well as domestic and international gross in sales.

What if you wanted to attach the movie title to each individual review?  LEFT JOIN will add that info from `movies` to every row of `ratings`.

In [26]:
# Join sales data to movies data
qs = """ SELECT * FROM ratings
          LEFT JOIN movies ON ratings.movieid = movies.movieid
          LIMIT 2;
          """
run_query(qs)

(['userid', 'movieid', 'rating', 'timestamp', 'movieid', 'title', 'genres'],
 [(64572,
   5,
   4.0,
   datetime.datetime(1997, 3, 27, 4, 14, 10),
   5,
   'Father of the Bride Part II (1995)',
   'Comedy'),
  (86695,
   5,
   4.0,
   datetime.datetime(1997, 6, 29, 16, 41, 44),
   5,
   'Father of the Bride Part II (1995)',
   'Comedy')])

### Selecting specific columns during a join

Here's another order of operations thing...  even though it looks like your data isn't joined until later, that's not strictly true based on order of operations.  What this means is that you can actually select columns from both tables in your SELECT statement.  This will let you keep only what you want from both tables.  

Let's run the same query as above but only get `userid, movieid, rating, timestamp, and title`. Note that just like we have to use the `table_name.column_name` format in our ON, we also need to use that in our SELECT to get just one of the `movieid` columns.

In [27]:
# Join movie data to ratings data
qs = """ SELECT userid, ratings.movieid, rating, timestamp, title FROM ratings
          LEFT JOIN movies ON ratings.movieid = movies.movieid
          LIMIT 2;
          """
run_query(qs)

(['userid', 'movieid', 'rating', 'timestamp', 'title'],
 [(64572,
   5,
   4.0,
   datetime.datetime(1997, 3, 27, 4, 14, 10),
   'Father of the Bride Part II (1995)'),
  (86695,
   5,
   4.0,
   datetime.datetime(1997, 6, 29, 16, 41, 44),
   'Father of the Bride Part II (1995)')])

**TRY IT OUT**  Just do a simple left join bringing the sales table on to the reviews table.

In [40]:
# Join sales data to ratings data
qs = """SELECT * FROM ratings
        LEFT JOIN movie_sales ON ratings.movieid = movie_sales.movieid;"""
len(run_query(qs)[1])

500

## Subqueries

In python you would commonly filter results by several elements in different columns.  For example, if you had a bunch of spotify data in a single table you might filter by artist name and tempo and number of listens.  It makes sense to do the same thing here in SQL land, but it's a touch trickier as often those data will be in different tables.  So, it makes it difficult if you want to retrieve data from one table that meets a condition that's in another table.  

For example, what if we want to get only the movie ratings for movies that are a comedy.  Ratings are in one table, `ratings`, but we want to filter by genre which is over in the `movies` table.  To do this we need to use a subquery.  Essentially we'll filter via WHERE `ratings` by the output of another query on `movies`.

Let me show you the whole thing and then I'll break it down a bit more.  

In [29]:
qs = """ SELECT * FROM ratings
            WHERE movieid IN (SELECT movieid FROM movies WHERE genres LIKE '%Comedy%')
            LIMIT 20;"""
run_query(qs)

(['userid', 'movieid', 'rating', 'timestamp'],
 [(64572, 5, 4.0, datetime.datetime(1997, 3, 27, 4, 14, 10)),
  (86695, 5, 4.0, datetime.datetime(1997, 6, 29, 16, 41, 44)),
  (151466, 5, 3.0, datetime.datetime(2017, 7, 3, 16, 13, 23)),
  (12966, 1, 4.5, datetime.datetime(2010, 4, 12, 7, 48, 22)),
  (123960, 7, 5.0, datetime.datetime(1999, 11, 2, 15, 1, 23)),
  (118164, 1, 4.0, datetime.datetime(2007, 8, 17, 22, 35, 40)),
  (10327, 1, 4.0, datetime.datetime(2000, 3, 18, 21, 16, 15)),
  (157060, 1, 5.0, datetime.datetime(2000, 11, 29, 20, 15, 46)),
  (90275, 3, 1.0, datetime.datetime(1999, 12, 23, 6, 16, 53)),
  (154467, 1, 4.0, datetime.datetime(2005, 2, 13, 7, 15, 32)),
  (114014, 1, 5.0, datetime.datetime(1999, 12, 29, 20, 44, 54)),
  (44608, 1, 3.0, datetime.datetime(1997, 3, 18, 9, 9, 54)),
  (54635, 5, 5.0, datetime.datetime(1997, 7, 3, 23, 5, 21)),
  (137626, 5, 1.0, datetime.datetime(2000, 12, 8, 16, 34, 38)),
  (149957, 3, 2.5, datetime.datetime(2007, 11, 19, 21, 38, 34)),
  (420

In [30]:
qs = """ SELECT movieid, rating, to_char(timestamp, 'MM-DD-YYYY') FROM ratings
            WHERE movieid IN (SELECT movieid FROM movies WHERE genres LIKE '%Comedy%')
            LIMIT 20;"""
run_query(qs)

(['movieid', 'rating', 'to_char'],
 [(5, 4.0, '03-27-1997'),
  (5, 4.0, '06-29-1997'),
  (5, 3.0, '07-03-2017'),
  (1, 4.5, '04-12-2010'),
  (7, 5.0, '11-02-1999'),
  (1, 4.0, '08-17-2007'),
  (1, 4.0, '03-18-2000'),
  (1, 5.0, '11-29-2000'),
  (3, 1.0, '12-23-1999'),
  (1, 4.0, '02-13-2005'),
  (1, 5.0, '12-29-1999'),
  (1, 3.0, '03-18-1997'),
  (5, 5.0, '07-03-1997'),
  (5, 1.0, '12-08-2000'),
  (3, 2.5, '11-19-2007'),
  (1, 5.0, '10-28-1996'),
  (7, 3.0, '12-03-2001'),
  (1, 3.5, '08-10-2016'),
  (1, 3.5, '11-29-2004'),
  (1, 4.0, '08-29-2001')])

I limited the return so it wasn't overly long. But, you can see that only a few of the movieid's are represented.  In this case, only those that are comedies.  

Let's take a second and run the subquery to show you what it would return.

In [31]:
qs = """ SELECT movieid FROM movies
            WHERE genres LIKE '%Comedy%' """
run_query(qs)

(['movieid'], [(1,), (3,), (4,), (5,), (7,)])

See, it gives us just those ID's where 'Comedy' appears in the genres column.  You could in theory hardcode this without the subquery.  For example, you could have used the following:

```
qs = """ SELECT * FROM ratings
            WHERE movieid IN (1, 3, 4, 5, 7);"""
```

The obvious problem here is that this isn't dynamic.  What if you want a different genre?  You'd have to go and first do a query in `movies` to find out all the ID's that match, then hardcode them in this second query.  And if your data changes you'd have to update it.  Or, if your data are even remotely long it's just not possible to hardcode.  Remember that this full dataset has 67k movies, of which likely 1/4 are comedies.  You can't reasonably hardcode that so you *must* use a subquery!

**TRY IT OUT**  Can you write a query that gets the titles of movies that made over 99 million dollars in domestic sales? (the numbers of not real, so don't get surprised :) )


In [34]:
#Question 3: Write your query as the answer
qs = """SELECT title FROM movies
        WHERE movieid IN (SELECT movieid FROM movie_sales WHERE domestic > 99000000);
          """
run_query(qs)

(['title'],
 [('Toy Story (1995)',), ('Jumanji (1995)',), ('GoldenEye (1995)',)])

Of course, you can have a more complicated subquery.  For example what if we want to get just the titles of movies where the average rating is 3.5 or greater.  We'll need to do a GROUP BY in our subquery.  

In [35]:
qs = """ SELECT title FROM movies
            WHERE movieid IN (SELECT movieid FROM ratings
                                GROUP By movieid
                                HAVING AVG(rating) >= 3.5) """
run_query(qs)

(['title'], [('Toy Story (1995)',), ('Heat (1995)',)])

# Feature generation

The last thing I'm going to show you is just that you can do more manipulations of columns in your select.  Specifically, you can do math with columns in SELECT. Obviously you can do this rapidly in python, but why bring in a bunch of columns to do simple math on when you can bring in just the resulting one.  

For example, below I get the total_sales by adding the domestic and international columns. I also alias it as 'total_sales'.  

In [36]:
# Add two columns plus an alias
qs = """ SELECT movieid, (domestic+international) as total_sales FROM movie_sales;
          """
run_query(qs)

(['movieid', 'total_sales'],
 [(1, 364543951),
  (2, 263758310),
  (3, 71518503),
  (4, 81347948),
  (5, 76578911),
  (6, 187433328),
  (7, 90000000),
  (8, 23837227),
  (9, 20214270),
  (10, 356429941)])